# Running a plugin that uses ImageJ1 windows
Running a plugin that uses ImageJ1 windows requires using ImageJ in GUI mode and requires handling the resulting windows.  Each plugin may have unique behavior to pay attention to.  E.g., the Rigid Registration plugin requires specific pixel types and operates on the currently selected window.

In [1]:
# The specified fiji install does not seem to ship with rigid registration plugin.  I directed it to my local install.  Just replace the path with the one to your own.
import imagej
#ij = imagej.init('sc.fiji:fiji')

# Rigid Registration does not have a headless version.  You need to specify that ImageJ will NOT be operating in headless mode.
ij = imagej.init('C:/Users/michael/Fiji.app', headless=False)

Added 383 JARs to the Java classpath.


WARNING 2019-11-12 12:06:16,245: JVM is already running, will not add endpoints to classpath -- required classes might not be on classpath..


In [2]:
from skimage import io
import numpy as np

In [3]:
import os
cwd = os.getcwd()

In [4]:
from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')

In [ ]:
original_img = io.imread('https://acdbio.com/sites/default/files/styles/sample_image/public/M-IL-8-8-hr_0.jpg')

In [ ]:
moving_img = np.mean(original_img, axis=2)

In [ ]:
# Let's make the fixed image a translated image so that we can see if the plugin works.
imshape = np.shape(moving_img)
fixed_img = np.zeros(imshape)
fixed_img[40:, 40:] = moving_img[0:(imshape[0]-40), 0:(imshape[1]-40)]

In [ ]:
# Rigid Registration only allows a few pixel types, such as 8-bit grayscale
ij.ui().show('fixed image', ij.py.to_java(fixed_img))
ij.py.run_macro("""run("8-bit");""")

# You need to specify that no changes have been made, or else the close window dialogue at the end will ask for confirmation.
img = WindowManager.getCurrentImage()
img.changes = False

ij.ui().show('moving image', ij.py.to_java(moving_img))
ij.py.run_macro("""run("8-bit");""")
img = WindowManager.getCurrentImage()
img.changes = False


In [ ]:
# Define the plugin parameters.  You can apply the registration to as many open images as you want.  Simple keep adding key/value pairs with 'Image name': True, as with the 'moving image': True below
plugin = 'Rigid Registration'

args = { 
    'initialtransform': [],
    'n': 1,
    'tolerance': 1.000,
    'level': 4,
    'stoplevel': 2,
    'materialcenterandbbox': [],
    'showtransformed': 1,
    'template': 'fixed image',
    'measure': 'Correlation',
    'moving image': True
}

In [ ]:
ij.py.run_plugin(plugin, args)

In [ ]:
# The window should be open in ImageJ if you are running this cell-by-cell; if not, here you can see the result.
result = WindowManager.getCurrentImage()
ij.py.show(result)

In [ ]:
# Get the numpy array out of the image.  You must do this before closing the windows.
result_array = ij.py.from_java(result)
result_array

In [ ]:
# Close the windows in Fiji.
WindowManager.closeAllWindows()